<a href="https://colab.research.google.com/github/ancestor9/2025_Winter_Deep-Learning-with-TensorFlow/blob/main/backpropagation_calculus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. 스컬라에 대하여**

- **수식**:

$$
y = x^2
$$

- **미분**:

$$
\frac{dy}{dx} = 2x
$$

- **값 대입**:

$$
x = 2 \;\Rightarrow\; \text{gradient} = 4
$$

👉 **`loss.backward()`는 수식을 보고 미분식을 자동 계산한 것**



In [4]:
import torch

# 1️⃣ 미분 대상 변수 선언
x = torch.tensor(2.0, requires_grad=True)

# 2️⃣ 간단한 수식 정의
y = x ** 2        # y = x²

# 3️⃣ backward 실행
y.backward()     # x에 대해 y( = x²)를 미분한다

# 4️⃣ 결과 확인
print(x.grad)    # dy/dx = 2x = 4

tensor(4.)


## **2. 손실함수(Loss function)형태 대하여**

- 모델: $y = wx$
- 손실: $L = (wx - y)^2$
- 미분 결과: $\frac{\partial L}{\partial w} = 2(wx - y)x$

👉 이 값이 자동으로 `w.grad`에 저장

In [19]:
import torch

# 파라미터 (학습 대상)
w = torch.tensor(1.0, requires_grad=True)

# 입력과 정답
x = torch.tensor(3.0)
y_true = torch.tensor(10.0)

# 모델
y_pred = w * x

# 손실 함수 (MSE)
loss = (y_pred - y_true) ** 2
print(loss)

# 역전파
loss.backward()

print(w.grad)

tensor(49., grad_fn=<PowBackward0>)
tensor(-42.)


In [17]:
print("current w:", w)

current w: tensor(1., requires_grad=True)


### **경사하강(gradient descent)**
- **Forward → loss 계산 → loss.backward() → w.grad 생성 → w 업데이트**

> $w_{\text{new}} = w_{\text{old}} - \alpha \frac{\partial L}{\partial w}$

> 1 - 0.01*(-42)

In [21]:
1 - 0.01*(-42)

1.42

In [18]:
learning_rate = 0.01

with torch.no_grad():
    w -= learning_rate * w.grad

print("updated w:", w)

updated w: tensor(1.4200, requires_grad=True)


In [23]:
# gradient 초기화
w.grad.zero_()

print("updated w:", w)


updated w: tensor(1., requires_grad=True)


$$
\mathbf{w}_{t+1}
=
\mathbf{w}_t
-
\alpha \nabla_{\mathbf{w}} L(\mathbf{w}_t)
$$


$$
\mathbf{W}_{t+1}
=
\mathbf{W}_t
-
\alpha \frac{\partial L}{\partial \mathbf{W}}
$$


## **3. 단순회귀식 적용**

In [24]:
# ==========================================
# 1. 라이브러리 로드
# ==========================================
import numpy as np                     # 수치 계산 라이브러리
import torch                           # PyTorch 핵심 라이브러리

# 텐서 출력 포맷 설정 (가독성 개선)
torch.set_printoptions(edgeitems=2, linewidth=75)


# ==========================================
# 2. 데이터 정의
# ==========================================

# t_c : 실제 측정된 섭씨 온도 (정답값)
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]

# t_u : 관측된 화씨 온도 (입력값)
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

# Python list → PyTorch Tensor 변환
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

print("t_c shape:", t_c.shape)
print("t_u:", t_u)


# ==========================================
# 3. 모델 정의 (선형 함수)
# ==========================================

def model(t_u, w, b):
    """
    선형 회귀 모델
    t_u : 입력 데이터
    w   : 가중치(weight)
    b   : 편향(bias)
    """
    return w * t_u + b


# ==========================================
# 4. 손실 함수 정의
# ==========================================

def loss_fn(t_p, t_c):
    """
    평균 제곱 오차 (MSE)
    t_p : 예측값
    t_c : 실제값
    """
    squared_diffs = (t_p - t_c) ** 2     # 오차 제곱
    return squared_diffs.mean()          # 평균


# ==========================================
# 5. 파라미터 초기화
# ==========================================

# 스칼라 텐서로 초기화
w = torch.ones((), requires_grad=True)  # 초기 가중치 = 1
b = torch.zeros((), requires_grad=True) # 초기 절편 = 0


# ==========================================
# 6. 학습 루프
# ==========================================

learning_rate = 1e-2    # 학습률
n_epochs = 5000         # 반복 횟수

for epoch in range(1, n_epochs + 1):

    # ----- (1) 순전파 -----
    t_p = model(t_u, w, b)               # 예측값 계산
    loss = loss_fn(t_p, t_c)             # 손실 계산

    # ----- (2) 역전파 -----
    loss.backward()                      # d(loss)/dw, d(loss)/db 계산

    # ----- (3) 파라미터 업데이트 -----
    with torch.no_grad():                # 그래프 추적 중단
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad

    # print(f'w와 b의 값은 : {w}, {b}')
    # print('*'*100)

    # ----- (4) gradient 초기화 -----
    w.grad.zero_()      # 이 것을  안하면 누적함
    b.grad.zero_()
    # print(f'w와 b의 값은 : {w}, {b}')

    # ----- (5) 로그 출력 -----
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss {loss.item():.4f}, w={w.item():.4f}, b={b.item():.4f}")

t_c shape: torch.Size([11])
t_u: tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000,
        21.8000, 48.4000, 60.4000, 68.4000])
Epoch 1000, Loss nan, w=nan, b=nan
Epoch 2000, Loss nan, w=nan, b=nan
Epoch 3000, Loss nan, w=nan, b=nan
Epoch 4000, Loss nan, w=nan, b=nan
Epoch 5000, Loss nan, w=nan, b=nan


## 🔴 NaN(Not a Number) 발생 원인 분석

- **결론**  
  결과값이 `nan`으로 출력된 이유는  
  **학습률(learning_rate)이 너무 높아 가중치와 손실값이 발산했기 때문**이다.

---

## 1️⃣ 학습률($1 \times 10^{-2}$)의 과도함

- **현상**
  - 입력 데이터 $t_u$의 값 범위가 **20 ~ 80**으로 큰 상태에서
  - 학습률을 $1e^{-2}$로 설정함
  - 한 번의 파라미터 업데이트 시 가중치가 **과도하게 변화**

- **결과**
  - 기울기(Gradient)가 급격히 커짐
  - 파라미터 값이 컴퓨터가 표현할 수 있는 범위를 초과
  - `inf (무한대)` → `nan (Not a Number)`로 변환

- **자료 근거**
  - 첨부된 노트북 **Page 3**
  - 학습률이 $1e^{-2}$일 때  
    **단 11 epoch 만에 `Loss: inf` 발생 확인**

---

## 2️⃣ 근본 원인 요약

- 입력 데이터의 **스케일이 큰 상태**
- 학습률이 **입력 스케일에 비해 과도**
- Gradient 폭주(Exploding Gradient) 발생
- 손실값과 가중치가 수치적으로 발산

---

## 3️⃣ 해결 방법: 데이터 정규화 (Normalization)

- **접근 방법**
  - 입력 데이터 $t_u$의 스케일을 조정
  - 예:  
    ```python
    t_u = t_u * 0.1
    ```

- **효과**
  - Gradient 크기 안정화
  - 손실 함수 정상 수렴
  - `nan` 문제 해결

- **노트북 적용 사례**
  - 첨부된 노트북에서는  
    **입력 데이터 스케일링을 통해 문제를 해결**하고 있음

---

## 🎯 핵심 요약

- `nan`은 코드 오류가 아니라 **수치적 불안정성의 신호**
- **입력 스케일 ↔ 학습률의 균형**이 핵심
- 딥러닝에서 문제 발생 시  
  👉 **가장 먼저 입력 데이터와 학습률을 점검해야 함**



In [25]:
# ----------------------------------
# 입력 스케일 조정
# ----------------------------------
t_u_norm = t_u * 0.1

w = torch.ones((), requires_grad=True)
b = torch.zeros((), requires_grad=True)

learning_rate = 1e-2
n_epochs = 5000

for epoch in range(1, n_epochs + 1):

    t_p = model(t_u_norm, w, b)
    loss = loss_fn(t_p, t_c)

    loss.backward()

    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad

    w.grad.zero_()
    b.grad.zero_()

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss {loss.item():.4f}, w={w.item():.4f}, b={b.item():.4f}")

Epoch 1000, Loss 3.8285, w=4.8021, b=-14.1031
Epoch 2000, Loss 2.9577, w=5.2644, b=-16.7200
Epoch 3000, Loss 2.9286, w=5.3489, b=-17.1980
Epoch 4000, Loss 2.9277, w=5.3643, b=-17.2853
Epoch 5000, Loss 2.9276, w=5.3671, b=-17.3012
